# Spyglass access tutorial
This tutorial demonstrates how to access and explore data from the Kind Lab datasets using Spyglass.

- Connect to the Spyglass database using DataJoint
- Query session, subject, and experimental metadata
- Access digital I/O events, task epochs, and electrode information
- Work with NWB files through the Spyglass framework

The example focuses on a specific experimental session (Baseline-tone-flash-hab) from subject Rat-1021.

---

## Import Required Libraries

In [23]:
from pathlib import Path
import datajoint as dj
dj.conn(use_tls=False)  # configurable in config using [datababse.use_tls]
dj_local_conf_path = "/home/alessandra/CatalystNeuro/kind-lab-to-nwb/dj_local_conf.json"
dj.config.load(dj_local_conf_path)  # load config for database connection info
# to prevent needing to load before import, consider saving the config
dj.config.save_global()  # default loaded for anywhere on the system
dj.config.save_local()  # loaded when running a script from this directory

# spyglass.common has the most frequently used tables
import spyglass.common as sgc  # this import connects to the database
# spyglass.data_import has tools for inserting NWB files into the database
import spyglass.data_import as sgi
# LFP Imports
import spyglass.lfp as sglfp
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename, get_nwb_file

from kind_lab_to_nwb.spyglass_utils import AnnotatedEvents

# Visualization
import matplotlib.pyplot as plt
# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

[2025-12-18 11:42:10,547][INFO]: DataJoint is configured from /home/alessandra/CatalystNeuro/kind-lab-to-nwb/dj_local_conf.json


## Spyglass database

In [24]:
sgc.Session()

nwb_file_name name of the NWB file,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
sub-1072-Grin2b(6)_ses-OR-HabD1_.nwb,1072-Grin2b(6),University of Edinburgh,Kind,OR_HabD1-2023-09-11,Experimental Day 1. Individual rat placed in arena for 10 min habituation,2023-09-11 13:58:59,2023-09-11 13:58:59,"This experiment is part of the Rat Behavioural Phenotyping Pipeline Methodologies https://sidb.org.uk/facilities/rat-behavioural-phenotyping-pipeline/. As an initial effort to characterize the rat lines, these models are being behaviorally phenotyped at SIDB. Rat models are phenotyped according to a rigorous pipeline that assesses behaviours relevant to autism with intellectual disability, such as social and motor skills, and learning and sensory processing. Different cohorts of rats are run through different subsets of tasks to overcome potential order effects on behaviours and to limit the number of tasks each animal is put through. This experiment investigated object recognition memory in different rat models (Arid1b: https://rgd.mcw.edu/rgdweb/report/strain/main.html?id=14394518, Grin2b: https://rgd.mcw.edu/rgdweb/report/strain/main.html?id=14394515, Scn2a: https://rgd.mcw.edu/rgdweb/report/strain/main.html?id=25394530). The experimenter was blind to genotype throughout the experiment and watched the trials on a screen from outside the testing room. The task utilized a custom-built arena (60x60 cm) with black laminated wood walls and grey lino floor. Rats underwent habituation (3 days, 10 min each) followed by sample and test trials to assess their ability to recognize novel objects. The protocol included sample trials (15 min with two identical objects) and test trials (3 min) with one novel object. Both short-term (5 min delay) and long-term (24 hr delay) memory were assessed. Object exploration was recorded via CCTV camera and manually scored using BORIS software."
sub-1073-Grin2b(6)_ses-OLM-HabD2_.nwb,1073-Grin2b(6),University of Edinburgh,Kind,OLM_HabD2-2023-08-22,Experimental Day 2. Individual rat placed in arena for 10 min habituation,2023-08-22 14:22:37,2023-08-22 14:22:37,"This experiment is part of the Rat Behavioural Phenotyping Pipeline Methodologies https://sidb.org.uk/facilities/rat-behavioural-phenotyping-pipeline/. As an initial effort to characterize the rat lines, these models are being behaviorally phenotyped at SIDB. Rat models are phenotyped according to a rigorous pipeline that assesses behaviours relevant to autism with intellectual disability, such as social and motor skills, and learning and sensory processing. Different cohorts of rats are run through different subsets of tasks to overcome potential order effects on behaviours and to limit the number of tasks each animal is put through. This experiment investigated object location memory in different rat models (Arid1b: https://rgd.mcw.edu/rgdweb/report/strain/main.html?id=14394518, Grin2b: https://rgd.mcw.edu/rgdweb/report/strain/main.html?id=14394515, Scn2a: https://rgd.mcw.edu/rgdweb/report/strain/main.html?id=25394530). The experimenter was blind to genotype throughout the experiment and watched the trials on a screen from outside the testing room. The task utilized two custom-built arenas (60x60 cm) with different wall colors and bedding. Rats underwent habituation followed by sample and test trials to assess their ability to detect object location changes. The protocol included group habituation (10 min), individual habituation (2 days, 10 min each), sample trials (15 min with two identical objects), and test trials (3 min) with one object moved to a new location. Both short-term (5 min delay) and long-term (1 hr delay) memory were assessed. Object exploration was recorded via CCTV camera and manually scored using BORIS software."
sub-1275-Grin2b(10)_ses-AFC-2-HabD2_.nwb,1275_Grin2b(10),University of Edinburgh,Kind,AFC_2_HabD2,"Experimental Day 2. Habituation to

In [25]:
# Load session data
session_id = "Baseline-tone-flash-hab"
subject_id = "Rat-1021"
nwb_file_name = get_nwb_copy_filename(f"sub-{subject_id}_ses-{session_id}.nwb")
sgc.Session & {'nwb_file_name': nwb_file_name}

nwb_file_name name of the NWB file,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,Rat_1021,University of Edinburgh,Kind,Baseline_tone_flash_hab,"Experimental day 4. Session to probe CS (blue led flash) pre-exposure without fear. On experimental day 4, recordings were made while a subset of 19 rats were pre-exposed to 10s of blue flashing light (5 Hz 110 lux flashes, 50 / 50 duty cycle) of the same properties as that used for subsequent conditioning and recall, the purpose of this exposure was to test for behavioral and LFP responses to the sensory stimuli prior to association with a footshock. Pre-exposure was followed immediately by conditioning.",2020-09-24 14:07:01,2020-09-24 14:07:01,"The experiment investigated fear conditioning in male wild-type and Syngap1+/Delta-GAP rats (n=31, ages 3-6 months). Recordings included Local Field Potentials (LFP), electroencephalogram (EEG), head-mounted accelerometer data, and behavioral video recordings across five experimental days. The protocol involved context habituation, seizure screening, and a fear conditioning paradigm where rats were exposed to blue flashing light (5 Hz, 110 lux) paired with foot shocks. Data were collected using OpenEphys software and a 16-channel Intan digitizing head stage. Behavioral paradigm triggers were managed using FreezeFrame software, and behavioral cameras recorded rat movements throughout the experimental sessions. The experimental design allowed for assessing neural responses, seizure occurrence, and fear learning in these genetic variants."


In [26]:
sgc.Subject() & {'subject_id': "Rat_1021"}

subject_id,age,description,genotype,sex,species
Rat_1021,P12W/P24W,"Subjects were Evans-SGem2/PWC were genotyped by PCR. This study used male rats (n=31, 16 wild-types and 15 Syngap+/âˆ†-GAP) bred in-house and maintained in standard housing on a 12h/12h light dark with ad libitum access to water and food. Experimental animals were weaned from their dams postnatal day 22 (P22D) and housed with their WT littermates, 2-4 animals per cage. Genetically modified males and WT littermates ranging from 3 to 6 months of age were used for all experiments.",WT,M,Rattus norvegicus


In [27]:
sgc.DIOEvents & {"nwb_file_name": nwb_file_name}

nwb_file_name name of the NWB file,dio_event_name the name assigned to this DIO event,dio_object_id the object id of the data in the NWB file,interval_list_name descriptive name of this interval list
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,TTL_audio_trigger,dfdb22b8-4061-41d1-bfa5-488b887ea83c,raw data valid times
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,TTL_LED_trigger,294bc6e8-917e-46c1-9eaa-fb72769573e0,raw data valid times


In [28]:
(sgc.DIOEvents & {"nwb_file_name": nwb_file_name, "dio_event_name": dio_event_name}).fetch_nwb()[0]

{'nwb_file_name': 'sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb',
 'dio_event_name': 'TTL_audio_trigger',
 'dio_object_id': 'dfdb22b8-4061-41d1-bfa5-488b887ea83c',
 'interval_list_name': 'raw data valid times',
 'dio': TTL_audio_trigger pynwb.base.TimeSeries at 0x138829300139680
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (1941641,), type "<f8">
   description: TTL signal that drives an audio tune played to the animal as a neutral stimulus
   offset: 0.0
   rate: 2000.0
   resolution: -1.0
   starting_time: 0.0
   starting_time_unit: seconds
   unit: n.a.}

In [29]:
sgc.TaskEpoch & {"nwb_file_name": nwb_file_name}

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),task_name,camera_name,interval_list_name descriptive name of this interval list,task_environment the environment the animal was in,camera_names list of keys corresponding to entry in CameraDevice
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,1,baseline_tone_flash_habituation,None,01,None,=BLOB=


In [30]:
sgc.ElectrodeGroup & {"nwb_file_name": nwb_file_name}

nwb_file_name name of the NWB file,electrode_group_name electrode group name from NWBFile,region_id,probe_id a unique ID for this probe & dynamic config,description description of electrode group,target_hemisphere
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,Aud_LFP_electrode_group,8,Intan C3334,LFP signal recorded from the Auditory Cortex.,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,bad_electrode_group,7,Intan C3334,Bad electrodes group.,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,CrusI_LFP_electrode_group,9,Intan C3334,LFP signal recorded from the Crus I.,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,HPC_EEG_electrode_group,4,Intan C3334,EEG signal recorded from the Hippocampus.,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,LA_LFP_electrode_group,12,Intan C3334,LFP signal recorded from the Lateral Amygdala.,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,M1_EEG_electrode_group,5,Intan C3334,EEG signal recorded from the Motor Cortex.,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,mPFC_LFP_electrode_group,11,Intan C3334,LFP signal recorded from the Medial Prefrontal Cortex (Infralimbic cortex).,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,OB_EEG_electrode_group,6,Intan C3334,EEG signal recorded from the Olfactory Bulb.,Right
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,V1_LFP_electrode_group,10,Intan C3334,LFP signal recorded from the Primary Visual Cortex.,Right


In [31]:
sgc.Electrode & {"nwb_file_name": nwb_file_name}

nwb_file_name name of the NWB file,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode,probe_id a unique ID for this probe & dynamic config,probe_shank unique shank number within probe.,"probe_electrode electrode ID, output from acquisition",region_id,name unique label for each contact,original_reference_electrode the configured reference electrode for this electrode,x the x coordinate of the electrode position in the brain,y the y coordinate of the electrode position in the brain,z the z coordinate of the electrode position in the brain,filtering description of the signal filtering,impedance electrode impedance,"bad_channel if electrode is ""good"" or ""bad"" as observed during recording",x_warped x coordinate of electrode position warped to common template brain,y_warped y coordinate of electrode position warped to common template brain,z_warped z coordinate of electrode position warped to common template brain,contacts label of electrode contacts used for a bipolar signal - current workaround
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,Aud_LFP_electrode_group,2,Intan C3334,2,2,8,2,2,nan,nan,nan,unfiltered,nan,False,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,Aud_LFP_electrode_group,3,Intan C3334,3,3,8,3,3,nan,nan,nan,unfiltered,nan,False,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,bad_electrode_group,1,Intan C3334,1,1,7,1,1,nan,nan,nan,unfiltered,nan,True,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,bad_electrode_group,4,Intan C3334,4,4,7,4,4,nan,nan,nan,unfiltered,nan,True,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,bad_electrode_group,7,Intan C3334,7,7,7,7,7,nan,nan,nan,unfiltered,nan,True,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,bad_electrode_group,13,Intan C3334,13,13,7,13,13,nan,nan,nan,unfiltered,nan,True,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,bad_electrode_group,14,Intan C3334,14,14,7,14,14,nan,nan,nan,unfiltered,nan,True,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,bad_electrode_group,15,Intan C3334,15,15,7,15,15,nan,nan,nan,unfiltered,nan,True,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,CrusI_LFP_electrode_group,12,Intan C3334,12,12,9,12,12,nan,nan,nan,unfiltered,nan,False,0.0,0.0,0.0,
sub-Rat-1021_ses-Baseline-tone-flash-hab_.nwb,HPC_EEG_electrode_group,11,Intan C3334,11,11,4,11,11,nan,nan,nan,unfiltered,nan,False,0.0,0.0,0.0,
